# Data Science for Business - Recurrent Neural Network (RNN) on Wine Reviews with TensorFlow

## Initialize notebook
Load required packages and set up workspace, e.g., initialize the random number generator.

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization
from sklearn import metrics
import matplotlib.pyplot as plt

In [2]:
np.random.seed(42)
tf.random.set_seed(42)

Check if we are running on GPU.

In [3]:
tf.config.experimental.list_physical_devices('GPU')

[]

# Problem description

The dataset contains 130.000+ wine reviews (Source: https://www.kaggle.com/datasets/zynicide/wine-reviews) written by professional sommeliers. Besides a text, it contains information about the wine, such as the country of origin, the variety, the winery, the price, and the number of points given by the reviewer. The goal is this notebook is to predict the number of points given by the reviewer based on the text of the review.

In [4]:
corpus = pd.read_csv("https://raw.githubusercontent.com/olivermueller/ds4b-2024/refs/heads/main/Session_10/winemag-data-130k-v2.csv")
corpus.head()

Split data into three sets: training, validation, and test.

In [5]:
training = corpus.iloc[0:80000,].sample(n=10000) # to speed up training
validation = corpus.iloc[80000:100000,]
test = corpus.iloc[100000:,]

For each dataset, store features and targets in separate variables

In [6]:
train_corpus_features = training[["description"]]
train_corpus_target = training[["points"]]
val_corpus_features = validation[["description"]]
val_corpus_target = validation[["points"]]
test_corpus_features = test[["description"]]
test_corpus_target = test[["points"]]

Next, we have to create [TensorFlow `Datasets`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) from the Pandas dataframes. The use of TensorFlow Datasets follows a common pattern:

1.   Create a dataset from raw data (e.g., a Pandas dataframe, a CSV file, multiple text files).
2.   Apply transformations to preprocess the data in the dataset (e.g., tokenize and vectorize the texts).
3. Iterate over the dataset and process its elements. Iteration happens in a streaming fashion, so the full dataset does not need to fit into memory.

Here, we use the `from_tensor_slices` constructor to create datasets from dataframes.

In [7]:
train_ds = tf.data.Dataset.from_tensor_slices((tf.cast(train_corpus_features.values, tf.string),
                                               tf.cast(train_corpus_target.values, tf.int32)))

val_ds = tf.data.Dataset.from_tensor_slices((tf.cast(val_corpus_features.values, tf.string),
                                             tf.cast(val_corpus_target.values, tf.int32)))

test_ds = tf.data.Dataset.from_tensor_slices((tf.cast(test_corpus_features.values, tf.string),
                                              tf.cast(test_corpus_target.values, tf.int32)))

Display some stats and examples from the created datasets.

In [8]:
for inputs, targets in train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("===")
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])
    break

inputs.shape: (1,)
inputs.dtype: <dtype: 'string'>
targets.shape: (1,)
targets.dtype: <dtype: 'int32'>
===
inputs[0]: tf.Tensor(b"This Cab-based Bordeaux blend is very dense in fruit and also very Tannic with a capital T. Such is the astringency that it may never go away before the blackberry and black currant fruit drops out. Nonetheless it's dry and savory and, in its own Rockpiley way, classic. Drink with superrich barbecue, well-marbled grilled beef and such. Tasted twice.", shape=(), dtype=string)
targets[0]: tf.Tensor(89, shape=(), dtype=int32)


# Vectorize documents

We will now use [TensorFlow's `TextVectorization`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/TextVectorization) function to transform raw texts into numerical vectors. To do this, we simply map words to integers (`output_mode = 'int'`). To get document representation of a fixed length, we limit the length of documents to 100 tokens (longer documents will be truncated, shorter documents will be padded with zeros). We also set a maximum vocabulary size of 10000 words.

In [9]:
max_tokens = 10000
max_length = 100

text_vectorization = TextVectorization(
    max_tokens = max_tokens,
    output_mode = "int",
    output_sequence_length = max_length
)

Some apects of the `TextVectorization` function (e.g., the size and contents of the vocabulary) have to be fit using training data, which can be done with the `adapt` function (which can only be applied to the features (x) of the dataset). 

In [10]:
train_ds_features_only = train_ds.map(lambda x, y: x)
text_vectorization.adapt(train_ds_features_only)

2025-01-08 21:15:53.393304: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Show the vocabulary that our vectorizer knows after being fit to the training data.

In [11]:
text_vectorization.get_vocabulary()[0:10]

['', '[UNK]', 'and', 'the', 'a', 'of', 'with', 'this', 'is', 'wine']

Now, we can apply our adapted `text_vectorization` function to all three datasets.

In [12]:
vectorized_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls = 4)

vectorized_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls = 4)

vectorized_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls = 4)

Show results.

In [13]:
for inputs, targets in vectorized_train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("===")
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])
    break

inputs.shape: (1, 100)
inputs.dtype: <dtype: 'int64'>
targets.shape: (1,)
targets.dtype: <dtype: 'int32'>
===
inputs[0]: tf.Tensor(
[   7 7069  598   52    8   63  144   11   16    2  114   63  104    6
    4 7052 8254  738    8    3  953   20   13  619 1067  557  853  406
    3   53    2   29  137   16 3846   82 1514   14   43    2  141    2
   11   14  643 9072  301  452   25    6 3294  952 7575  367  500    2
  738 1385 3276    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0], shape=(100,), dtype=int64)
targets[0]: tf.Tensor(89, shape=(), dtype=int32)


We can use `get_vocabulary()` to get the word behind a specific index.

In [14]:
text_vectorization.get_vocabulary()[1385]

'tasted'

# Train model

We are now ready to specify a neural network and feed it with the vectroized datasets. For convenience, we define a custome function `get_model` which defines the network architecture, creates a model from it, and compiles this model (by defining, e.g., an otpimizer and loss function).

Instead of averaging or flattening the outputs of the embedding layer, a RNN/LSTM layer can directly process its 2D output (i.e., it takes a sequence of vectors as input instead of a single vector).

In [15]:
def get_model(hidden_dim=32):
    inputs = keras.Input(shape=(max_length,), dtype="int64")
    embedded = layers.Embedding(input_dim=max_tokens, output_dim=300, mask_zero=True)(inputs)
    hidden1 = layers.LSTM(hidden_dim, return_sequences = False)(embedded)
    outputs = layers.Dense(1, activation = "linear")(hidden1)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer = tf.optimizers.Adam(),
                  loss = "mean_absolute_error",
                  metrics = ["mean_absolute_error"])
    return model

Instantiate model and show it's architecture.

In [16]:
model = get_model()
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 100, 300)  │  3,000,000 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, 100)       │          0 │ input_layer[0][0] │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 32)        │     42,624 │ embedding[0][0],  │
│                     │                   │            │ not_equal[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 1)         │         33 │ lstm[0][0]        │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,042,657 (11.61 MB)

 Trainable params: 3,042,657 (11.61 MB)

 Non-trainable params: 0 (0.00 B)

Fit model on training data and save best model to disk. We use the validation set to monitor how the performance of the model evolves during training.

In [ ]:
callbacks = [keras.callbacks.ModelCheckpoint("lstm.keras", save_best_only=True)]

history = model.fit(vectorized_train_ds.cache(),
          validation_data = vectorized_val_ds.cache(),
          epochs = 3,
          batch_size = 128,
          callbacks = callbacks)

Epoch 1/3
 3721/10000 ━━━━━━━━━━━━━━━━━━━━ 3:15 31ms/step - loss: 51.1082 - mean_absolute_error: 51.1082

Plot the learning process.

In [ ]:
plt.plot(history.history['mean_absolute_error'])
plt.plot(history.history['val_mean_absolute_error'])
plt.title('Model accuracy')
plt.ylabel('Mean Absolute Error')
plt.xlabel('Epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

# Make predictions

Load best model from training phase.

In [33]:
model = keras.models.load_model("mlp.keras")

Make predictions on test set.

In [ ]:
preds = model.predict(vectorized_test_ds)

Calculate prediction error.

In [ ]:
print(metrics.mean_absolute_error(test_corpus_target, preds))